In [1]:
!pip install -q --upgrade langchain langchain_google_genai langchain-core langchain_community docs2txt pypdf langchain_chroma sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 631.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.4 MB/s eta 

##**What is Retrieval Augmented Generation (RAG)?**







###**LangChain Components and Expression Language (LCEL)**
1. Large Language Model (LLM)
2. Output Parsers
3. Structured Output
4. Prompt Templates
5. LLM Messages

In [ ]:
import os
os.environ["GOOGLE_API_KEY"]=""

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ['LANGCHAIN_API_KEY']=""
os.environ['LANGCHAIN_PROJECT']=''

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_response=llm.invoke("Tell me a simple joke about coding!")
llm_response

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run-466d6a03-dd88-4ce6-8248-c4b4adc820a0-0', usage_metadata={'input_tokens': 8, 'output_tokens': 14, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}})

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()
output_parser.invoke(llm_response)

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [6]:
chain=llm | output_parser
chain.invoke("Tell me a story!")

"Elara, a clockwork raven with emerald eyes, perched on the spire of the Grand Obsidian Clocktower.  Her gears whirred softly, a counterpoint to the booming chime that announced the hour – midnight.  Below, the city of Aethelburg glittered, a tapestry woven with gaslight and shadow.  Elara wasn't like the other clockwork creations; she possessed a spark of something more, a flicker of sentience that the master clockmaker, old Silas, had accidentally imbued into her during her creation.\n\nSilas, a recluse with a beard like tangled wires, had tasked Elara with a vital duty: guarding the Chronarium, a heart-shaped crystal housed within the tower that regulated the city's time.  Without it, Aethelburg would descend into temporal chaos.\n\nTonight, however, something felt different.  A tremor, not in the earth, but in the very fabric of time, rippled through the tower.  Elara's internal mechanisms sputtered, her emerald eyes widening.  From the shadows below, a figure emerged – a cloaked w

In [7]:
from typing import List
from pydantic import BaseModel,Field

class ReviewDSA(BaseModel):
  topic:str=Field(description="Name of the DSA topic")
  summary: str=Field(description="Brief summary of the review")
  rating: float=Field(description="Overall rating out of 5")
  pros: List[str]=Field(description="List of positive aspects")


prompt_text='''
 Just started learning Dynamic Programming, and wow, this topic is mind-blowing! The way it optimizes
    recursive solutions is just amazing. It really helps in solving problems efficiently that involve
    overlapping subproblems, like Fibonacci and Knapsack.

    But not gonna lie, it's tough at first. You really need to practice a lot to get the hang of thinking
    in terms of subproblems. Also, memorization techniques can be confusing, and it’s easy to mess up
    the transition from recursion to tabulation.

    Overall, I'd rate it a 4.5 out of 5. Once you get it, it’s a game-changer for problem-solving.
    Definitely worth the effort!
'''

structured_llm=llm.with_structured_output(ReviewDSA)
output=structured_llm.invoke(prompt_text)
print(output)
print(output.pros)

topic='"Dynamic Programming"' summary='"Mind-blowing topic, but tough at first. Requires a lot of practice and can be confusing at times."' rating=4.5 pros=['"Optimizes recursive solutions"', '"Efficient for overlapping subproblems"', '"Game-changer for problem-solving"']
['"Optimizes recursive solutions"', '"Efficient for overlapping subproblems"', '"Game-changer for problem-solving"']


In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt.invoke({"topic":"sports"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a joke about sports', additional_kwargs={}, response_metadata={})])

In [10]:
chain=prompt | llm|output_parser
result=chain.invoke({"topic":"coding"})
print(result)

Why do programmers prefer dark mode?

Because light attracts bugs!


In [13]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="You are a helpful coding assistant!"),
    HumanMessage(content="Tell me a joke about Debugging in code")
]

response=llm.invoke(messages)
print(response)

content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run-a7f51658-77e7-4607-902f-01546818b4c5-0' usage_metadata={'input_tokens': 15, 'output_tokens': 14, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}


In [20]:
template=ChatPromptTemplate([
    ("system","You are a helpful coding assistant"),
    ("human","tell me a joke about {topic}")
])

chain=template | llm
result=chain.invoke({"topic":"coding"})
print(result)

content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run-a2c90a6b-3d08-485e-8299-a57a86de21da-0' usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


In [21]:
!pip install Docx2txt

###**Document Processing**
- Loading Documents
- Splitting Documents

In [23]:
from langchain_community.document_loaders import PyPDFLoader,Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,length_function=len)

docx_loader=Docx2txtLoader("/content/docs/SUPRATIMNAG_AIML.docx")
documents=docx_loader.load()


splits=text_splitter.split_documents(documents)
print(f"splitted with {len(splits)} chunks")
print(len(documents))


splitted with 10 chunks
1


In [24]:
documents[0]

Document(metadata={'source': '/content/docs/SUPRATIMNAG_AIML.docx'}, page_content='Supratim Nag\n\nHowrah,West Bengal, India\n\n\uf10b (+91) 8420945255 | snsupratim@gmail.com | https://github.com/snsupratim | snsupratim-portfolio |  https://www.linkedin.com/in/snsupratim\uf08c\n\n\n\nSummary ---------------------------------------------------------------------------------\n\n\n\nEnthusiastic and detail-oriented B.Tech student in Computer Science and Engineering with a specialization in Artificial Intelligence and Machine Learning. Actively learning machine learning, deep learning, NLP, and generative AI techniques. Also worked with various tech stack to built full-stack web application .\n\n\n\nSupratim Nag\n\nHowrah,West Bengal, India\n\n\uf10b (+91) 8420945255 | snsupratim@gmail.com | https://github.com/snsupratim | snsupratim-portfolio |  https://www.linkedin.com/in/snsupratim\uf08c\n\n\n\nSummary ---------------------------------------------------------------------------------\n\n\

In [25]:
splits[0]

Document(metadata={'source': '/content/docs/SUPRATIMNAG_AIML.docx'}, page_content='Supratim Nag\n\nHowrah,West Bengal, India\n\n\uf10b (+91) 8420945255 | snsupratim@gmail.com | https://github.com/snsupratim | snsupratim-portfolio |  https://www.linkedin.com/in/snsupratim\uf08c\n\n\n\nSummary ---------------------------------------------------------------------------------\n\n\n\nEnthusiastic and detail-oriented B.Tech student in Computer Science and Engineering with a specialization in Artificial Intelligence and Machine Learning. Actively learning machine learning, deep learning, NLP, and generative AI techniques. Also worked with various tech stack to built full-stack web application .\n\n\n\nSupratim Nag\n\nHowrah,West Bengal, India\n\n\uf10b (+91) 8420945255 | snsupratim@gmail.com | https://github.com/snsupratim | snsupratim-portfolio |  https://www.linkedin.com/in/snsupratim\uf08c\n\n\n\nSummary ---------------------------------------------------------------------------------')

In [27]:
def load_documents(folder_path : str)-> List[Document]:
  documnets=[]
  for filename in os.listdir(folder_path):
    file_path=os.path.join(folder_path,filename)
    if filename.endswith('.pdf'):
      loader=PyPDFLoader(file_path)
    elif filename.endswith('.docx'):
      loader=Docx2txtLoader(file_path)
    else:
      print(f"unsupported file type : {filename}")
      continue
    documents.extend(loader.load())
  return documents

folder_path='/content/docs'
documents=load_documents(folder_path)
print(f"loaded {len(documents)} from folder")

splits=text_splitter.split_documents(documents)
print(f"splitted documents into {len(splits)} chunks")


loaded 5 from folder
splitted documents into 30 chunks


In [28]:
documents[1]

Document(metadata={'producer': '', 'creator': 'WPS Writer', 'creationdate': '2024-12-30T16:33:57+05:30', 'author': 'SUPRATIM NAG', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-12-30T16:33:57+05:30', 'sourcemodified': "D:20241230163357+05'30'", 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/docs/cv-aiml-intern.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Supratim Nag\nHowrah,West Bengal, India\n(+91) 8420945255 | snsupratim@gmail.com | https://github.com/snsupratim | snsupratim-portfolio | https://www.linkedin.com/in/snsupratim\nObjective -------------------------------------------------------------------\nEnthusiastic and detail-oriented B.Tech student in Computer Science and Engineering with a specialization in Artificial\nIntelligence and Machine Learning. Actively learning machine learning, deep learning, NLP, and generative AI\ntechniques. Seeking an internship to apply my technical skills, enhance my knowledge, and c

In [29]:
splits[4]

Document(metadata={'source': '/content/docs/SUPRATIMNAG_AIML.docx'}, page_content='Tools :                                 Git, GitHub, Docker, Jupyter , HuggingFace, MLflow, Colab, Cursor\n\nDatabase  :                         MongoDB, Firebase, PostgreSQL, MySQL , SupaBase\n\nPlatforms :                             Render, AWS ,Vercel\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProjects   ------------------------------------------------------------------------------------- \n\n\n\nCrop-Sisease Detection\n\nGoogle Colab,\n\nDETECTED CROP DISEASE USING PYTORCH & RESNET-50\n\nFlask,MERN Stack \n\n• Built an web application to detect crop disease prediction by uploading disease images & participated in IIT-BHU Eco-Hackthon .\n\n\n\nGenerative AI\n\nPython,Flask\n\nBuild an AI Chatbot with LLMs ( Gemini)\n\nStreamlit,Render\n\n• An AI Chatbot where user can  provide their own knowledge base to train the llms as per their requirement.\n\n\n\nProjects   --------------------------------------


###**Creating Embeddings**
- Using GoogleAI Embeddings
- Using SentenceTransformer

In [31]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
document_embedding=embedding.embed_documents([split.page_content for split in splits])
print(f"created embedding for {len(document_embedding)} documents chunks")

created embedding for 30 documents chunks



###**Setting Up the Vector Store**
- Creating the Vector Store
- Performing Similarity Search
- Creating a Retriever

In [34]:
from langchain_chroma import Chroma

embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
collection_name="my_collection"
vectorstore=Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)

print("vector store created and persisted to './chroma_db' ")

vector store created and persisted to './chroma_db' 


In [36]:
query="what is the resume summary?"

search_results=vectorstore.similarity_search(query,k=2)
print(f"\n Top 2 most relevant search for the query : '{query}' \n ")
for i,result in enumerate(search_results,1):
  print(f"result {i}:")
  print(f"source: {result.metadata.get('source','Unknown')}")
  print(f"content : {result.page_content}")
  print()


 Top 2 most relevant search for the query : 'what is the resume summary?' 
 
result 1:
source: /content/docs/SUPRATIMNAG_AIML.docx
content : Summary ---------------------------------------------------------------------------------



Enthusiastic and detail-oriented B.Tech student in Computer Science and Engineering with a specialization in Artificial Intelligence and Machine Learning. Actively learning machine learning, deep learning, NLP, and generative AI techniques. Also worked with various tech stack to built full-stack web application .











Education ----------------------------------------------------------------------------------



MCKV Institute of Engineering 

Howrah,West Bengal, India 

B.Tech in CSE with specializing in AI/ML

October 2022  June 2026* 

• 9.56/10 (3rd  year 2nd sem SGPA)

• Relevant Course : Machine Learning , Artificial Intelligence , High Performance Computing , NVIDIA CUDA ,

Cloud Computing , Data Mining , SpringBoot ,Hibernate etc.



Educati

In [37]:
retriever=vectorstore.as_retriever(search_kwargs={"k":2})
retriever_results=retriever.invoke("who is the candidate whose cv we are seeing?")
print(retriever_results)

[Document(id='a1c1ac47-2cc4-4842-8ff8-09c59d51a548', metadata={'source': '/content/docs/SUPRATIMNAG_AIML.docx'}, page_content='Summary ---------------------------------------------------------------------------------\n\n\n\nEnthusiastic and detail-oriented B.Tech student in Computer Science and Engineering with a specialization in Artificial Intelligence and Machine Learning. Actively learning machine learning, deep learning, NLP, and generative AI techniques. Also worked with various tech stack to built full-stack web application .\n\n\n\n\n\n\n\n\n\n\n\nEducation ----------------------------------------------------------------------------------\n\n\n\nMCKV Institute of Engineering \n\nHowrah,West Bengal, India \n\nB.Tech in CSE with specializing in AI/ML\n\nOctober 2022  June 2026* \n\n• 9.56/10 (3rd  year 2nd sem SGPA)\n\n• Relevant Course : Machine Learning , Artificial Intelligence , High Performance Computing , NVIDIA CUDA ,\n\nCloud Computing , Data Mining , SpringBoot ,Hibernat








###**Building the RAG Chain**
- Creating the RAG Chain
- Using the RAG Chain

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template="""
Answer the question based only on the following context:
{context}
Question:{question}
Answer: """


prompt=ChatPromptTemplate.from_template(template)

def docs2str(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain=(
    {"context":retriever | docs2str,"question":RunnablePassthrough()}
    | prompt
    | llm
    |StrOutputParser()
)


In [39]:
question="which college does the person studies?"
response=rag_chain.invoke(question)
print(f"question: {question}")
print(f"response: {response}")

question: which college does the person studies?
response: MCKV Institute of Engineering



###**Handling Follow-Up Questions**
- Creating a History-Aware Retriever
- Using the History-Aware RAG Chain

In [42]:
from decimal import Context
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain


context_q_system_prompt="""
Given a chat history and latest user question
 which might refernce context in chat history,
formulate a standalone quesion which can be understood without the chat history.
Do not answer the question,
just formulate it if needed and otherwise return as it is.
"""

context_q_prompt=ChatPromptTemplate.from_messages([
    ("system",context_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}"),
])

context_chain=context_q_prompt | llm | StrOutputParser()
print(context_chain.invoke({"input":"what does his cgpa? ","chat_history":[]}))

What is his CGPA?


In [43]:
from langchain.chains import create_retrieval_chain

history_aware_retriever=create_history_aware_retriever(
    llm,retriever,context_q_prompt
)

qa_prompt=ChatPromptTemplate.from_messages([
     ("system","you are a helpfull AI assistant. use following context to answer the user's question."),
     ("system","Context:{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}"),
])

question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [45]:
from langchain_core.messages import HumanMessage,AIMessage

chat_history=[]
question1="where was his college?"
answer1=rag_chain.invoke({"input":question1,"chat_history":chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(question1)
print(answer1)

where was his college?
The student attended MCKV Institute of Engineering in Howrah, West Bengal, India.


In [48]:
question2="what was his sgpa and his college name?"
answer2=rag_chain.invoke({"input":question2,"chat_history":chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(question2)
print(answer2)

what was his sgpa and his college name?
His most recent SGPA (Semester Grade Point Average) was 9.56/10.  His college is MCKV Institute of Engineering in Howrah, West Bengal, India.
